<a href="https://colab.research.google.com/github/Sajishvar/Spell_Grammer_Checker_Tamil/blob/sajishvar/Spell_Grammer_Checker_Tamil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from collections import Counter

# Adjust encoding configuration for standard output in environments supporting it.
if hasattr(sys.stdout, 'reconfigure'):
    sys.stdout.reconfigure(encoding='utf-8')

In [ ]:
def load_dictionary(file_path):
    # Load words from a dictionary file
    with open(file_path, 'r', encoding='utf-8') as file:
        dictionary = set(line.strip() for line in file)
    return dictionary

In [ ]:
def word_length_similarity(misspelt_word, candidate_word):
    # Binary score: 1.0 if lengths match, 0.0 if they don't
    return 1.0 if len(misspelt_word) == len(candidate_word) else 0.0

In [ ]:
def character_frequency_similarity(misspelt_word, candidate_word):
    # Count frequency of each character in both words
    misspelt_counter = Counter(misspelt_word)
    candidate_counter = Counter(candidate_word)
    # Compute sum of minimum matches for each character
    matching_count = sum(min(misspelt_counter[char], candidate_counter[char]) for char in misspelt_counter)
    # Normalize by the length of the misspelt word
    return matching_count / len(misspelt_word)